In [1]:
# Step 1: Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score



In [2]:
# Step 2: Load the Iris Dataset
df = pd.read_csv('Iris.csv')

In [5]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [14]:
 df.shape

(150, 6)

In [6]:
# Step 3: Prepare Features and Target
X = df.drop("Species", axis=1)
y = df["Species"]

# Encode target labels (since it's a classification problem)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [7]:
# Step 4: Split into Train and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Step 5: Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [9]:
# Step 6: Function to Train Model and Return Loss
def train_model(learning_rate):
    model = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=learning_rate, max_iter=1000, tol=1e-3, random_state=42)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_val)
    loss = log_loss(y_val, y_prob)
    return loss

In [10]:
# Step 7: Bisection/Ternary Search to Find Optimal Learning Rate
def find_best_learning_rate(low=1e-6, high=1.0, tolerance=1e-4, max_iter=25):
    for _ in range(max_iter):
        mid1 = low + (high - low) / 3
        mid2 = high - (high - low) / 3

        loss1 = train_model(mid1)
        loss2 = train_model(mid2)

        if loss1 < loss2:
            high = mid2
        else:
            low = mid1

        if abs(high - low) < tolerance:
            break

    return (low + high) / 2


In [11]:
# Step 8: Estimate Optimal Learning Rate
best_lr = find_best_learning_rate()
print(f"🔍 Best learning rate found: {best_lr:.6f}")



🔍 Best learning rate found: 0.999955


In [12]:
# Step 9: Train Final Model with Best Learning Rate
final_model = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=best_lr, max_iter=1000, tol=1e-3, random_state=42)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)



In [13]:
# Step 10: Accuracy Check
accuracy = accuracy_score(y_val, y_pred)
print(f"✅ Validation Accuracy using best learning rate: {accuracy:.4f}")

✅ Validation Accuracy using best learning rate: 1.0000
